In [49]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

## Loading Graphs
* Multigraph with
    - Edge features
        - h : A list of features in data
        - Label (0-1)
        - Attack
    - Node features
        - {1,.., 1} same length as h

In [50]:
file_path = "test.graph"
# Use dgl.save_graphs to save the graph to the specified file
##dgl.save_graphs(file_path, [test_g])
# Use dgl.load_graphs to load the graph from the file
test_g, _ = dgl.load_graphs(file_path)
# The loaded_graphs variable now contains the loaded DGLGraph(s)
test_g = test_g[0]  # Assuming you saved a single graph

# Specify the path to the saved graph file
file_path = "train.graph"
# Use dgl.save_graphs to save the graph to the specified file
##dgl.save_graphs(file_path, [train_g])
# Use dgl.load_graphs to load the graph from the file
train_g, _ = dgl.load_graphs(file_path)
# The loaded_graphs variable now contains the loaded DGLGraph(s)
train_g = train_g[0]  # Assuming you saved a single graph

In [51]:
import joblib
lab_enc = joblib.load('gnn_label_encoder.pkl')

# Self-Supervised Learning
### E-GraphSAGE

In [52]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      """
      Reset parameters whenever object created
      """
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      """
      It sends the 'h' feature data from edges to nodes
      """
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      """
      update_all : message aggregation
      applies a linear transformation
      concatenates node features with aggregated neighbor features
      then applies a non-linear activation function (ReLU)
      """
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [53]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      """
      If corruption : permutate edge features
      Then send data into layers to find node&edge features
      """
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        efeats = efeats[e_perm]
      for i, layer in enumerate(self.layers):
        nfeats, e_feats = layer(g, nfeats, efeats)
      return nfeats.sum(1), e_feats.sum(1)

# Self-Supervised Learning
### Deep Graph Infomax (DGI)

In [54]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
        super(Discriminator, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
        self.reset_parameters()

    def uniform(self, size, tensor):
        bound = 1.0 / math.sqrt(size)
        if tensor is not None:
            tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
        size = self.weight.size(0)
        self.uniform(size, self.weight)

    def forward(self, features, summary):
        features = torch.matmul(features, torch.matmul(self.weight, summary))
        return features

In [55]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
        super(DGI, self).__init__()
        self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
        #self.discriminator = Discriminator(128)
        self.discriminator = Discriminator(256)
        self.loss = nn.BCEWithLogitsLoss()

    # def forward(self, graph, feat, e_features, edge_weight=None, embed=False):
    #     feat = torch.reshape(train_g.ndata['h'],
    #                                (train_g.ndata['h'].shape[0], 1,
    #                                 39))
    #     positive = self.encoder(graph, feat, e_features, corrupt=False)
    #     negative = self.encoder(graph, feat, e_features, corrupt=True)
    #     self.loss = nn.BCEWithLogitsLoss()

    def forward(self, graph, feat, e_features, edge_weight=None, eweight=None, embed=False):
 
        feat = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    39))
        positive = self.encoder(graph, feat, e_features, corrupt=False)
        negative = self.encoder(graph, feat, e_features, corrupt=True)
        if embed:
            return e_features
            #return feat

        positive = positive[1]
        negative = negative[1]

        summary = torch.sigmoid(positive.mean(dim=0))

        positive = self.discriminator(positive, summary)
        negative = self.discriminator(negative, summary)

        l1 = self.loss(positive, torch.ones_like(positive))
        l2 = self.loss(negative, torch.zeros_like(negative))

        return torch.tensor([[l1+l2, 0]],requires_grad=True)

## Training DGI
* Same hyperparameters and optimizer specified in the "Anomal-E".

In [56]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [57]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [58]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [59]:
# Convert to GPU
train_g = train_g

In [60]:
node_features = train_g.ndata['h']
edge_features = train_g.edata['h']


## Loading trained DGI
* Same hyperparameters and optimizer specified in the "Anomal-E".

In [61]:
dgi.load_state_dict(torch.load('best_dgi.pkl'))

<All keys matched successfully>

In [62]:
dgi(train_g, node_features, edge_features)

tensor([[5.9682, 0.0000]], requires_grad=True)

In [63]:
train_g

Graph(num_nodes=64237, num_edges=2641554,
      ndata_schemes={'h': Scheme(shape=(1, 39), dtype=torch.float32)}
      edata_schemes={'Label': Scheme(shape=(), dtype=torch.int64), 'Attack': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1, 39), dtype=torch.float32)})

## Edge Embeddings


* Training DGI 
* Seperate encoders are trained for training and testing graph
* After encoding train and test graphs, encodings are converted into dataframe

In [64]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [65]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [66]:
# Convert to GPU
test_g = test_g

In [67]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [68]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [69]:
df_train # Edge features, labels

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.213101,0.084576,-0.100087,-0.085792,-0.504479,-0.286598,0.044959,-0.109970,-0.071820,-0.931622,...,-0.443378,-0.176202,-0.041914,-0.069238,0.232846,-0.652440,-0.520690,-0.528282,Benign,0
1,0.213101,0.084576,-0.100087,-0.085792,-0.504479,-0.286598,0.044959,-0.109970,-0.071820,-0.931622,...,-0.443378,-0.176202,-0.041914,-0.069238,0.232846,-0.652440,-0.520690,-0.528282,Benign,0
2,0.213101,0.084576,-0.100087,-0.085792,-0.504479,-0.286598,0.044959,-0.109970,-0.071820,-0.931622,...,-0.443378,-0.176202,-0.041914,-0.069238,0.232846,-0.652440,-0.520690,-0.528282,Benign,0
3,0.213101,0.084576,-0.100087,-0.085792,-0.504479,-0.286598,0.044959,-0.109970,-0.071820,-0.931622,...,-0.443378,-0.176202,-0.041914,-0.069238,0.232846,-0.652440,-0.520690,-0.528282,Benign,0
4,0.213101,0.084576,-0.100087,-0.085792,-0.504479,-0.286598,0.044959,-0.109970,-0.071820,-0.931622,...,-0.443378,-0.176202,-0.041914,-0.069238,0.232846,-0.652440,-0.520690,-0.528282,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641549,0.238272,0.086273,-0.082979,-0.075177,-0.517827,-0.300409,0.021976,-0.122864,-0.048900,-0.925022,...,-0.459370,-0.149649,-0.044086,-0.073823,0.225230,-0.659248,-0.506145,-0.530789,Benign,0
2641550,0.254560,0.067924,-0.087581,-0.067362,-0.560228,-0.311366,0.002567,-0.094462,-0.053286,-0.910474,...,-0.451368,-0.077642,-0.046995,-0.065274,0.201917,-0.667047,-0.512167,-0.542341,Benign,0
2641551,0.254018,0.076960,-0.065283,-0.077068,-0.562148,-0.313146,-0.011261,-0.106118,-0.058118,-0.904931,...,-0.464071,-0.099481,-0.034149,-0.059856,0.233140,-0.666527,-0.509573,-0.542750,Benign,0
2641552,0.254433,0.075125,-0.074659,-0.076764,-0.565004,-0.303394,-0.004606,-0.099647,-0.059363,-0.909203,...,-0.454214,-0.091502,-0.042996,-0.059958,0.221726,-0.664430,-0.513076,-0.544611,Benign,0


In [70]:
df_test

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.212190,0.084235,-0.100871,-0.085811,-0.504105,-0.286591,0.045796,-0.109333,-0.072740,-0.931824,...,-0.442794,-0.176718,-0.041757,-0.069147,0.232927,-0.652265,-0.521252,-0.528130,Benign,0
1,0.212190,0.084235,-0.100871,-0.085811,-0.504105,-0.286591,0.045796,-0.109333,-0.072740,-0.931824,...,-0.442794,-0.176718,-0.041757,-0.069147,0.232927,-0.652265,-0.521252,-0.528130,Benign,0
2,0.212190,0.084235,-0.100871,-0.085811,-0.504105,-0.286591,0.045796,-0.109333,-0.072740,-0.931824,...,-0.442794,-0.176718,-0.041757,-0.069147,0.232927,-0.652265,-0.521252,-0.528130,Benign,0
3,0.212190,0.084235,-0.100871,-0.085811,-0.504105,-0.286591,0.045796,-0.109333,-0.072740,-0.931824,...,-0.442794,-0.176718,-0.041757,-0.069147,0.232927,-0.652265,-0.521252,-0.528130,Benign,0
4,0.212190,0.084235,-0.100871,-0.085811,-0.504105,-0.286591,0.045796,-0.109333,-0.072740,-0.931824,...,-0.442794,-0.176718,-0.041757,-0.069147,0.232927,-0.652265,-0.521252,-0.528130,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132126,0.238991,0.080186,-0.089852,-0.066152,-0.511611,-0.311789,0.026515,-0.120675,-0.042586,-0.925042,...,-0.460239,-0.139137,-0.044112,-0.079347,0.211982,-0.662015,-0.504174,-0.528458,Benign,0
1132127,0.254403,0.075197,-0.073904,-0.075690,-0.563776,-0.306023,-0.005386,-0.100197,-0.058240,-0.908402,...,-0.455886,-0.091104,-0.041516,-0.060288,0.221319,-0.665003,-0.511931,-0.543905,Benign,0
1132128,0.226226,0.082544,-0.094559,-0.076982,-0.505063,-0.296730,0.037320,-0.120199,-0.053299,-0.929006,...,-0.452944,-0.162573,-0.045611,-0.076051,0.224357,-0.657492,-0.512367,-0.527756,Benign,0
1132129,0.242786,0.084904,-0.082050,-0.070827,-0.520388,-0.306714,0.018430,-0.121979,-0.045771,-0.923596,...,-0.461895,-0.140351,-0.043430,-0.075043,0.219978,-0.661053,-0.503455,-0.531070,Benign,0


## GNNExplainer


In [71]:
df_gxai = pd.DataFrame(columns = ["GNNExplainer","PGExplainer"])

In [72]:
node_features

tensor([[[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        ...,

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 1., 1.,  ..., 1., 1., 1.]]])

In [73]:
import dgl.function as fn
import torch
import torch.nn as nn
from dgl.data import GINDataset
from dgl.dataloading import GraphDataLoader
from dgl.nn import AvgPooling, GNNExplainer, PGExplainer, SubgraphX

In [74]:
# Explain the prediction for graph 0
explainer = GNNExplainer(dgi, num_hops=2)

In [75]:
g = train_g
features = g.ndata['h']
edge_features = g.edata['h']
gnn_kwargs = {
    'e_features': edge_features,  
}

f = torch.reshape(features,(train_g.ndata['h'].shape[0], 39))


feat_mask, edge_mask = explainer.explain_graph(g, f, **gnn_kwargs)

Explain graph: 100%|████████████████████████████████████████████████████████████████████| 100/100 [09:07<00:00,  5.48s/it]


In [76]:
train_g

Graph(num_nodes=64237, num_edges=2641554,
      ndata_schemes={'h': Scheme(shape=(1, 39), dtype=torch.float32)}
      edata_schemes={'Label': Scheme(shape=(), dtype=torch.int64), 'Attack': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1, 39), dtype=torch.float32)})

In [77]:
feat_mask

tensor([0.2770, 0.3077, 0.2718, 0.2520, 0.2886, 0.2828, 0.2772, 0.3014, 0.2459,
        0.2853, 0.2685, 0.2600, 0.2857, 0.2692, 0.2498, 0.2726, 0.2771, 0.2444,
        0.2447, 0.2650, 0.2609, 0.2735, 0.2820, 0.2724, 0.2987, 0.2798, 0.2646,
        0.2961, 0.2863, 0.2457, 0.2877, 0.2747, 0.3031, 0.2793, 0.2692, 0.2736,
        0.2414, 0.2730, 0.2847])

In [78]:
edge_mask

tensor([0.2767, 0.2751, 0.2768,  ..., 0.2756, 0.2758, 0.2753])

In [79]:
len(feat_mask)

39

In [80]:
len(edge_mask)

2641554

In [81]:
edge_mask.shape

torch.Size([2641554])

In [82]:
df_gxai['GNNExplainer'] = edge_mask.numpy()

In [83]:
df_gxai.head(5)

,GNNExplainer,PGExplainer
0,0.276744,NaN
1,0.275142,NaN
2,0.276803,NaN
3,0.276444,NaN
4,0.274406,NaN


In [37]:
dgi(g, features, edge_features)[0].log_softmax(dim=-1)

tensor([-2.5556e-03, -5.9708e+00], grad_fn=<LogSoftmaxBackward0>)

In [38]:
train_g.ndata['h'].shape

torch.Size([64237, 1, 39])

In [39]:
torch.topk(edge_mask.flatten(), 5).indices

tensor([ 288332,  182919, 1012392,  247800, 1949184])

## PGExplainer


In [40]:
g = train_g
features = g.ndata['h']
edge_features = g.edata['h']
gnn_kwargs = {
    'e_features': edge_features,  
}

f = torch.reshape(features,(train_g.ndata['h'].shape[0], 39))


In [41]:
dgi.encoder

SAGE(
  (layers): ModuleList(
    (0): SAGELayer(
      (W_apply): Linear(in_features=78, out_features=128, bias=True)
      (W_edge): Linear(in_features=256, out_features=256, bias=True)
    )
  )
)

In [42]:
explainer = PGExplainer(dgi, num_features=39, num_hops=2)

probs, edge_weight = explainer.explain_graph(g, f, **gnn_kwargs, training=True)

In [43]:
edge_weight

tensor([0.2507, 0.3694, 0.4155,  ..., 0.6880, 0.4402, 0.4575],
       grad_fn=<DivBackward0>)

In [44]:
len(set(edge_weight.detach().numpy()))

2170986

In [45]:
df_gxai['PGExplainer'] = edge_weight.detach().numpy()

In [46]:
df_gxai

,GNNExplainer,PGExplainer
0,0.275427,0.250723
1,0.274867,0.369388
2,0.277122,0.415487
3,0.276152,0.379018
4,0.274712,0.328536
...,...,...
2641549,0.276421,0.594326
2641550,0.275020,0.639202
2641551,0.275678,0.688026
2641552,0.274718,0.440159


In [47]:
df_gxai.to_parquet("graph_explainers_df.parquet")

## SubgraphX

In [ ]:
# explainer = SubgraphX(dgi, num_hops=1)

In [ ]:
# ## Subgraph
# #graph = dgl.node_subgraph(train_g, random.sample(range(0, train_g.num_nodes()+1), 100))

# g = train_g
# features = g.ndata['h']
# edge_features = g.edata['h']
# l = torch.tensor(0)

# gnn_kwargs = {
#     'e_features': edge_features,
# }

# f = torch.reshape(features,(g.ndata['h'].shape[0], 39))

In [ ]:
# g_nodes_explain = explainer.explain_graph(g, f, target_class=l, **gnn_kwargs)